In [245]:
import itertools
import numpy as np
import pandas as pd
from scipy import constants
from functools import partial

In [246]:
file_name = "benzene.xyz"
with open(file_name) as file:
    file_data = []
    for line in file:
        line = line.strip()
        file_data.append(line)

    atom_count = int(file_data[0])
    print(file_data[1])
    file_data = file_data[2:]

    data = map(lambda x: x.split(), file_data)
    data = map(lambda x: (x[0], np.array([float(x[1]), float(x[2]), float(x[3])])), data)
    data = list(data)

    for atom, (x, y, z) in data:
        print(f" {atom:<3} \t {x:<8.3} \t {y:<8.3} \t {z:<8.3}")

data : list[tuple[str, float, float, float]]

benzene
 C   	 -0.269   	 -0.462   	 -1.43   
 C   	 0.0131   	 0.0539   	 -0.166  
 C   	 -0.24    	 -0.712   	 0.971   
 C   	 -0.775   	 -1.99    	 0.844   
 C   	 -1.06    	 -2.51    	 -0.421  
 C   	 -0.803   	 -1.74    	 -1.56   
 H   	 -1.02    	 -2.15    	 -2.54   
 H   	 -0.0714  	 0.135    	 -2.32   
 H   	 0.43     	 1.05     	 -0.0669 
 H   	 -0.0206  	 -0.311   	 1.96    
 H   	 -0.972   	 -2.59    	 1.73    
 H   	 -1.47    	 -3.51    	 -0.52   


In [247]:
def mag(v: np.ndarray):
    return np.sqrt(v @ v)

cov_rads = {  'H' : 0.37, 'C' : 0.77, 'O' : 0.73, 'N' : 0.75, 'F' : 0.71,
  'P' : 1.10, 'S' : 1.03, 'Cl': 0.99, 'Br': 1.14, 'I' : 1.33, 'He': 0.30,
  'Ne': 0.84, 'Ar': 1.00, 'Li': 1.02, 'Be': 0.27, 'B' : 0.88, 'Na': 1.02,
  'Mg': 0.72, 'Al': 1.30, 'Si': 1.18, 'K' : 1.38, 'Ca': 1.00, 'Sc': 0.75,
  'Ti': 0.86, 'V' : 0.79, 'Cr': 0.73, 'Mn': 0.67, 'Fe': 0.61, 'Co': 0.64,
  'Ni': 0.55, 'Cu': 0.46, 'Zn': 0.60, 'Ga': 1.22, 'Ge': 1.22, 'As': 1.22,
  'Se': 1.17, 'Kr': 1.03, 'X' : 0.00}

def is_bonded(ai, v : np.ndarray, aj, u : np.ndarray = np.array([0,0,0]), k=1.20):
    return mag(v-u) <= k * ( cov_rads.get(ai) + cov_rads.get(aj))

def bond_angle(v : np.ndarray, u : np.ndarray, w : np.ndarray):
    a = v-u 
    b = w-u
    
    cos = a @ b / mag(a) / mag(b)
    return np.arccos(cos)

In [248]:
bond_graph = [[] for _ in range(atom_count)]
bonds = []


for (i, (ai, vi)), (j, (aj, vj)) in itertools.combinations(enumerate(data), 2):
    if i == j:
        continue

    if is_bonded(ai,vi,aj,vj):
        bond_graph[i].append(j)
        bond_graph[j].append(i)
        bonds.append((i,j, mag(vi-vj)))

print("Bond Graph")
for i, bonded_atoms in enumerate(bond_graph):
    print(f" {i+1}\t {data[i][0]} \t {list(map(lambda x: x+1, bonded_atoms))}")
print()

print(f"{len(bonds)} bond(s) found (Angstrom)")
for i,j,d in bonds:
    if i < j:
        print(f" {i+1}-{j+1} \t ({data[i][0]}-{data[j][0]}) \t {d:.5}")

Bond Graph
 1	 C 	 [2, 6, 8]
 2	 C 	 [1, 3, 9]
 3	 C 	 [2, 4, 10]
 4	 C 	 [3, 5, 11]
 5	 C 	 [4, 6, 12]
 6	 C 	 [1, 5, 7]
 7	 H 	 [6]
 8	 H 	 [1]
 9	 H 	 [2]
 10	 H 	 [3]
 11	 H 	 [4]
 12	 H 	 [5]

12 bond(s) found (Angstrom)
 1-2 	 (C-C) 	 1.3948
 1-6 	 (C-C) 	 1.3948
 1-8 	 (C-H) 	 1.0867
 2-3 	 (C-C) 	 1.3948
 2-9 	 (C-H) 	 1.0867
 3-4 	 (C-C) 	 1.3948
 3-10 	 (C-H) 	 1.0867
 4-5 	 (C-C) 	 1.3948
 4-11 	 (C-H) 	 1.0867
 5-6 	 (C-C) 	 1.3948
 5-12 	 (C-H) 	 1.0867
 6-7 	 (C-H) 	 1.0867


In [249]:
angle_count = sum(map(lambda a: len(a) * (len(a)- 1) // 2, bond_graph))

print(f"{angle_count} angle(s) found (degrees)")
for j, bonded_atoms in enumerate(bond_graph):
    _, vj = data[j]
    for i,k in itertools.combinations(bonded_atoms, 2):
        _, vi = data[i]
        _, vk = data[k]
        
        angle_rad = bond_angle(vi, vj, vk)
        angle = np.rad2deg(angle_rad)

        print(f" {f'{i+1}-{j+1}-{k+1}':<10} {f'({data[i][0]}-{data[j][0]}-{data[k][0]})':<10} \t {np.round(angle, 1)}")

18 angle(s) found (degrees)
 2-1-6      (C-C-C)    	 120.0
 2-1-8      (C-C-H)    	 120.0
 6-1-8      (C-C-H)    	 120.0
 1-2-3      (C-C-C)    	 120.0
 1-2-9      (C-C-H)    	 120.0
 3-2-9      (C-C-H)    	 120.0
 2-3-4      (C-C-C)    	 120.0
 2-3-10     (C-C-H)    	 120.0
 4-3-10     (C-C-H)    	 120.0
 3-4-5      (C-C-C)    	 120.0
 3-4-11     (C-C-H)    	 120.0
 5-4-11     (C-C-H)    	 120.0
 4-5-6      (C-C-C)    	 120.0
 4-5-12     (C-C-H)    	 120.0
 6-5-12     (C-C-H)    	 120.0
 1-6-5      (C-C-C)    	 120.0
 1-6-7      (C-C-H)    	 120.0
 5-6-7      (C-C-H)    	 120.0


In [250]:
def torsion_angle(a : np.ndarray,b: np.ndarray,c: np.ndarray,d:np.ndarray):
    n1 = np.cross((a-b), (c-b))
    n2 = np.cross((b-c), (d-c))

    cos = n1 @ n2 / mag(n1) / mag(n2)
    return np.arccos(cos)

torsion_count = 0
torions = []

for j, k, _ in bonds:
    for i in bond_graph[j]:
        for l in bond_graph[k]:
            if len({i,j,k,l}) != 4:
                continue
            torsion_count += 1
            torions.append((i,j,k,l))

print(f"{torsion_count} torsion(s) found (degrees)")
for i,j,k,l in torions:
    _, vi = data[i]
    _, vj = data[j]
    _, vk = data[k]
    _, vl = data[l]

    angle = torsion_angle(vi, vj ,vk ,vl)
    print(f" {i+1}-{j+1}-{k+1}-{l+1} \t {data[i][0]}-{data[j][0]}-{data[k][0]}-{data[l][0]} \t {np.round(np.rad2deg(angle), 1)}")

24 torsion(s) found (degrees)
 6-1-2-3 	 C-C-C-C 	 0.0
 6-1-2-9 	 C-C-C-H 	 180.0
 8-1-2-3 	 H-C-C-C 	 180.0
 8-1-2-9 	 H-C-C-H 	 0.0
 2-1-6-5 	 C-C-C-C 	 nan
 2-1-6-7 	 C-C-C-H 	 180.0
 8-1-6-5 	 H-C-C-C 	 180.0
 8-1-6-7 	 H-C-C-H 	 0.0
 1-2-3-4 	 C-C-C-C 	 0.0
 1-2-3-10 	 C-C-C-H 	 180.0
 9-2-3-4 	 H-C-C-C 	 180.0
 9-2-3-10 	 H-C-C-H 	 0.0
 2-3-4-5 	 C-C-C-C 	 0.0
 2-3-4-11 	 C-C-C-H 	 180.0
 10-3-4-5 	 H-C-C-C 	 180.0
 10-3-4-11 	 H-C-C-H 	 0.0
 3-4-5-6 	 C-C-C-C 	 0.0
 3-4-5-12 	 C-C-C-H 	 180.0
 11-4-5-6 	 H-C-C-C 	 180.0
 11-4-5-12 	 H-C-C-H 	 0.0
 4-5-6-1 	 C-C-C-C 	 0.0
 4-5-6-7 	 C-C-C-H 	 180.0
 12-5-6-1 	 H-C-C-C 	 180.0
 12-5-6-7 	 H-C-C-H 	 0.0


C:\Users\jayz0\AppData\Local\Temp\ipykernel_20632\2355494410.py:6: RuntimeWarning: invalid value encountered in arccos
  return np.arccos(cos)


In [251]:
def out_of_plane_angle(o :np.ndarray, v:np.ndarray, u:np.ndarray, w:np.ndarray):
    n = np.cross((v-o), (u-o))
    n /= mag(n) 
    sin = n @ (w-o) / mag(w-o)
    return np.arcsin(sin)

out_of_plane_angle_count = 3 * sum(map(lambda x: len(x) == 3, bond_graph))
print(f"{out_of_plane_angle_count} out of plane angle(s) found")

for i,bonded_atoms in enumerate(bond_graph):
    if len(bonded_atoms) != 3:
        continue

    for j, k, l in itertools.permutations(bonded_atoms):
        if j < k: continue
        _, o = data[i]
        _, v = data[j]
        _, u = data[k]
        _, w = data[l]

        angle = out_of_plane_angle(o,v,u,w)

        print(f" ({j+1}-{i+1}-{k+1})-{l+1} \t ({data[i][0]}-{data[j][0]}-{data[k][0]})-{data[l][0]} \t {np.round(np.rad2deg(angle),2)}")



18 out of plane angle(s) found
 (6-1-2)-8 	 (C-C-C)-H 	 -0.0
 (8-1-2)-6 	 (C-H-C)-C 	 0.0
 (8-1-6)-2 	 (C-H-C)-C 	 -0.0
 (3-2-1)-9 	 (C-C-C)-H 	 -0.0
 (9-2-1)-3 	 (C-H-C)-C 	 0.0
 (9-2-3)-1 	 (C-H-C)-C 	 -0.0
 (4-3-2)-10 	 (C-C-C)-H 	 0.0
 (10-3-2)-4 	 (C-H-C)-C 	 -0.0
 (10-3-4)-2 	 (C-H-C)-C 	 0.0
 (5-4-3)-11 	 (C-C-C)-H 	 0.0
 (11-4-3)-5 	 (C-H-C)-C 	 -0.0
 (11-4-5)-3 	 (C-H-C)-C 	 0.0
 (6-5-4)-12 	 (C-C-C)-H 	 -0.0
 (12-5-4)-6 	 (C-H-C)-C 	 0.0
 (12-5-6)-4 	 (C-H-C)-C 	 -0.0
 (5-6-1)-7 	 (C-C-C)-H 	 0.0
 (7-6-1)-5 	 (C-H-C)-C 	 -0.0
 (7-6-5)-1 	 (C-H-C)-C 	 0.0


In [252]:
at_masses = {    'H' : 1.00794, 'C' : 12.0107, 'O' : 15.9994, 'N' : 14.0067,
  'F' : 18.9984, 'P' : 30.9738, 'S' : 32.0650, 'Cl': 35.4530, 'Br': 79.9040,
  'I' : 126.904, 'He': 4.00260, 'Ne': 20.1797, 'Ar': 39.9480, 'Li': 6.94100,
  'Be': 9.01218, 'B' : 10.8110, 'Na': 22.9898, 'Mg': 24.3050, 'Al': 26.9815,
  'Si': 28.0855, 'K' : 39.0983, 'Ca': 40.0780, 'Sc': 44.9559, 'Ti': 47.8670,
  'V' : 50.9415, 'Cr': 51.9961, 'Mn': 54.9380, 'Fe': 55.8450, 'Co': 58.9332,
  'Ni': 58.6934, 'Cu': 63.5460, 'Zn': 65.4090, 'Ga': 69.7230, 'Ge': 72.6400,
  'As': 74.9216, 'Se': 78.9600, 'Kr': 83.7980, 'X' : 0.00000}

In [253]:
mass = sum(map(lambda x: at_masses[x[0]], data))

print(f"molecular mass: {np.round(mass,2)}")

center_of_mass = sum(map(lambda x: x[1] * at_masses[x[0]], data), [0,0,0]) / mass
x,y,z = center_of_mass
print("molecular center of mass (Angstrom)")
print(f"(X,Y,Z): \t {x:.5} \t {y:.5} \t {z:.5}")

molecular mass: 78.11
molecular center of mass (Angstrom)
(X,Y,Z): 	 -0.52165 	 -1.228 	 -0.2936


In [254]:
moment_of_inertia = sum(map(lambda x: at_masses[x[0]] * ((x[1]-center_of_mass) @ (x[1]-center_of_mass) * np.identity(3) - np.outer(x[1]-center_of_mass, x[1]-center_of_mass)), data), [[0,0,0],[0,0,0],[0,0,0]])
print("moment of inertia matrix from the center of mass (amu * A^2)")
pd.DataFrame(moment_of_inertia, columns=list("XYZ"), index=list("XYZ"))

moment of inertia matrix from the center of mass (amu * A^2)


,X,Y,Z
X,164.394194,-31.151761,-4.150844
Y,-31.151761,101.547274,1.709070
Z,-4.150844,1.709070,88.950962


In [255]:
adjusted_data  : list[tuple[str, float, float, float]] = list(map(lambda x: (x[0], x[1] - center_of_mass) , data))

print("data after shifting center of mass")
for atom, (x, y, z) in adjusted_data:
    print(f" {atom:<3} \t {x:<8.3} \t {y:<8.3} \t {z:<8.3}")

data after shifting center of mass
 C   	 0.253    	 0.766    	 -1.14   
 C   	 0.535    	 1.28     	 0.127   
 C   	 0.282    	 0.516    	 1.27    
 C   	 -0.253   	 -0.766   	 1.14    
 C   	 -0.535   	 -1.28    	 -0.127  
 C   	 -0.282   	 -0.516   	 -1.27   
 H   	 -0.501   	 -0.917   	 -2.25   
 H   	 0.45     	 1.36     	 -2.02   
 H   	 0.951    	 2.28     	 0.227   
 H   	 0.501    	 0.917    	 2.25    
 H   	 -0.45    	 -1.36    	 2.02    
 H   	 -0.951   	 -2.28    	 -0.227  


In [272]:
moment_of_inertia : np.ndarray[np.ndarray[float]]
Ia, Ib, Ic = sorted(map(partial(np.round, decimals=3), np.linalg.eigvals(moment_of_inertia)))



print("principle moments of inertia (amu * A^2)")
print(f" {Ia} \t {Ib} \t {Ic}")
print()

print("Type of Molecule")

if Ia == 0 and Ib == 0 and Ic == 0:
    print(" Monoatomic")
elif Ia == Ib and Ib == Ic:
    print("Sphereical Top")
elif Ia == 0 and Ib == Ic:
    print(" Linear")
elif Ia == Ib:
    print(" Oblate Symmetric Top")
elif Ib == Ic:
    print(" Prolate Symmetric Top")
else:
    print(" Asymmetric Top")



principle moments of inertia (amu * A^2)
 88.723 	 88.723 	 177.446

Type of Molecule
 Oblate Symmetric Top
